In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
feature_extraction.py

Pipeline de extracción de cuatro métricas de conectividad a partir
de señales de ROIs AAL3, con adaptación para:
  - eliminación automática de ROIs vacías
  - corrección en la discretización de NMI para evitar índices fuera de rango
"""

import os
import logging
import numpy as np
import pandas as pd
import torch
import scipy.io as sio

from scipy.stats import zscore
from statsmodels.tsa.stattools import grangercausalitytests
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

# -------------------------------------------------------------------
# CONFIGURATION & PATHS
# -------------------------------------------------------------------
base_dir      = "/home/diego/Escritorio/ALL_170"
csv_path      = os.path.join(base_dir, "SubjectsDataAndTests.csv")
roi_dir       = os.path.join(base_dir, "ROISignals")
output_dir    = os.path.join(base_dir, "TensorData")
os.makedirs(output_dir, exist_ok=True)

# Sliding window for DCV (segundos)
dcv_window_sec = 60
dcv_step_sec   = 3
sampling_rate  = 1/3.0  # Hz (TR de 3s)

logging.basicConfig(
    filename='feature_extraction.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

df = pd.read_csv(csv_path)
subject_ids = df["SubjectID"].astype(str).tolist()

# -------------------------------------------------------------------
# UTILITIES
# -------------------------------------------------------------------
def _safe_div(a, b, eps=1e-8):
    return a / (b + eps)

# -------------------------------------------------------------------
# 1) CORRELATION with Fisher z
# -------------------------------------------------------------------
def compute_correlation_matrix(signals: np.ndarray) -> np.ndarray:
    z    = zscore(signals, axis=0, nan_policy='omit')
    corr = np.corrcoef(z, rowvar=False)
    corr = np.clip(corr, -0.999999, 0.999999)
    return 0.5 * np.log((1 + corr) / (1 - corr))

# -------------------------------------------------------------------
# 2) NORMALISED MUTUAL INFORMATION
# -------------------------------------------------------------------
def compute_nmi_matrix(signals: np.ndarray, num_bins: int = None) -> np.ndarray:
    T, N = signals.shape
    if num_bins is None:
        num_bins = int(np.clip(np.sqrt(T), 8, 16))
    print(f"    → T = {T} muestras; B = {num_bins} bins para MI")
    logging.info(f"compute_nmi: T={T}, bins={num_bins}")

    # percentiles por canal
    edges = np.percentile(signals, np.linspace(0, 100, num_bins + 1), axis=0).T  # (N, bins+1)
    disc = np.zeros((T, N), dtype=int)
    for i in range(N):
        # descartamos el límite mínimo y máximo
        bin_edges = edges[i][1:-1]
        # discretización en [0..num_bins-1]
        disc[:, i] = np.digitize(signals[:, i], bin_edges)

    # entropías marginales (añadimos pseudocuenta +1)
    ent = np.zeros(N)
    bias = (num_bins - 1) / (2 * T)
    for i in range(N):
        counts = np.bincount(disc[:, i], minlength=num_bins) + 1
        p = counts / counts.sum()
        H_emp = -np.sum(p * np.log(p))
        ent[i] = max(H_emp - bias, 0.0)

    # MI conjunta
    mi = np.zeros((N, N))
    bias_xy = (num_bins**2 - 1) / (2 * T)
    for i in range(N):
        for j in range(i+1, N):
            joint = np.ones((num_bins, num_bins), dtype=float)
            for t in range(T):
                joint[disc[t, i], disc[t, j]] += 1
            pxy = joint / joint.sum()
            Hxy = -np.sum(pxy * np.log(pxy))
            Iij = ent[i] + ent[j] - max(Hxy - bias_xy, 0.0)
            mi[i, j] = mi[j, i] = Iij

    denom = np.sqrt(np.outer(ent, ent)) + 1e-12
    return np.clip(mi / denom, 0.0, 1.0)

# -------------------------------------------------------------------
# 3) GRANGER CAUSALITY (parallel)
# -------------------------------------------------------------------
def _granger_worker(signals, i, j, max_lag):
    if i == j:
        return i, j, 0.0
    try:
        res = grangercausalitytests(signals[:, [i, j]], maxlag=max_lag, verbose=False)
        p   = res[max_lag][0]['ssr_ftest'][1]
        return i, j, -np.log(p) if p > 0 else 0.0
    except Exception as e:
        logging.error(f"GC error {i},{j}: {e}")
        return i, j, 0.0

def compute_gc_matrix(signals: np.ndarray, max_lag=1, n_workers=8) -> np.ndarray:
    N  = signals.shape[1]
    gc = np.zeros((N, N))
    args = ((signals, i, j, max_lag) for i in range(N) for j in range(N))
    with ProcessPoolExecutor(max_workers=n_workers) as exe:
        futures = [exe.submit(_granger_worker, *arg) for arg in args]
        for f in as_completed(futures):
            i, j, val = f.result()
            gc[i, j] = val
    return gc

# -------------------------------------------------------------------
# 4) DYNAMIC CONNECTIVITY VARIANCE (DCV)
# -------------------------------------------------------------------
def compute_dcv_matrix(signals: np.ndarray, window_sec, step_sec, sr) -> np.ndarray:
    T, N = signals.shape
    w    = int(window_sec * sr)
    s    = int(step_sec * sr)
    if w < 2:
        raise ValueError("Ventana DCV demasiado pequeña")
    mats = []
    for start in range(0, T - w + 1, s):
        seg = signals[start:start+w, :]
        mats.append(compute_correlation_matrix(seg))
    var = np.var(np.stack(mats, axis=2), axis=2)
    return np.sqrt(var)

# -------------------------------------------------------------------
# 5) PROCESS SINGLE SUBJECT
# -------------------------------------------------------------------
def process_subject(subject_id: str):
    mat_file = os.path.join(roi_dir, f"ROISignals_{subject_id}.mat")
    data     = sio.loadmat(mat_file)
    sig      = data.get('ROISignals', data.get('signals', None))
    if sig is None:
        raise KeyError(f"No signals in {subject_id}")

    # 1) Detectar y eliminar ROIs vacías (todo NaN o todo 0)
    nan_mask  = np.isnan(sig).all(axis=0)
    zero_mask =    (sig == 0.0).all(axis=0)
    bad_cols  = np.where(nan_mask | zero_mask)[0].tolist()
    if bad_cols:
        print(f"[{subject_id}] ➔ columnas vacías detectadas y eliminadas: {bad_cols}")
        logging.info(f"{subject_id}: empty ROIs removed {bad_cols}")
        sig = np.delete(sig, bad_cols, axis=1)
    else:
        print(f"[{subject_id}] ➔ no se detectaron columnas vacías.")

    # 2) Sustituir NaN/Inf residuales
    sig = np.nan_to_num(sig, nan=0.0, posinf=0.0, neginf=0.0)
    T, N = sig.shape
    print(f"[{subject_id}] señal tras limpieza: T={T}, N={N}")
    logging.info(f"{subject_id}: signals shape after clean {sig.shape}")

    # Cálculo de métricas
    try:
        corr = compute_correlation_matrix(sig)
        print(f" corr: {corr.shape}")
    except Exception as e:
        print(f" ⚠️ Error en CORR: {e}")
        corr = np.zeros((N, N))

    try:
        mi = compute_nmi_matrix(sig)
        print(f" nmi: {mi.shape}")
    except Exception as e:
        print(f" ⚠️ Error en NMI: {e}")
        mi = np.zeros((N, N))

    try:
        gc = compute_gc_matrix(sig)
        print(f" gc: {gc.shape}")
    except Exception as e:
        print(f" ⚠️ Error en GC:  {e}")
        gc = np.zeros((N, N))

    try:
        dcv = compute_dcv_matrix(sig, dcv_window_sec, dcv_step_sec, sampling_rate)
        print(f" dcv: {dcv.shape}")
    except Exception as e:
        print(f" ⚠️ Error en DCV: {e}")
        dcv = np.zeros((N, N))

    # Apilar y poner diagonal a 0
    tensor_np = np.stack([corr, mi, gc, dcv], axis=0)
    for c in range(4):
        np.fill_diagonal(tensor_np[c], 0.0)

    # Guardado
    grp  = df.loc[df['SubjectID'].astype(str)==subject_id, 'ResearchGroup'].values[0]
    grp  = grp if grp in ['AD','CN'] else 'Other'
    path = os.path.join(output_dir, f"{grp}_tensor_{subject_id}.pt")
    torch.save(torch.tensor(tensor_np, dtype=torch.float32), path)
    print(f"[{subject_id}] guardado en {path}")
    logging.info(f"{subject_id}: saved {path}")
    return path

# -------------------------------------------------------------------
# 6) MAIN
# -------------------------------------------------------------------
if __name__ == '__main__':
    for sid in tqdm(subject_ids, desc="Extracting features"):
        try:
            process_subject(sid)
        except Exception as e:
            logging.error(f"Subject {sid} failed: {e}")



/home/diego/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Extracting features:   0%|          | 0/432 [00:00<?, ?it/s]

[002_S_0295] ➔ columnas vacías detectadas y eliminadas: [34, 35, 80, 81]
[002_S_0295] señal tras limpieza: T=140, N=166
 corr: (166, 166)
    → T = 140 muestras; B = 11 bins para MI
 nmi: (166, 166)


Extracting features:   0%|          | 1/432 [00:08<1:00:16,  8.39s/it]

 gc: (166, 166)
 dcv: (166, 166)
[002_S_0295] guardado en /home/diego/Escritorio/ALL_170/TensorData/CN_tensor_002_S_0295.pt
[002_S_0413] ➔ columnas vacías detectadas y eliminadas: [34, 35, 80, 81]
[002_S_0413] señal tras limpieza: T=140, N=166
 corr: (166, 166)
    → T = 140 muestras; B = 11 bins para MI
 nmi: (166, 166)


Extracting features:   0%|          | 2/432 [00:16<59:59,  8.37s/it]  

...

 gc: (166, 166)
 dcv: (166, 166)
[305_S_6899] guardado en /home/diego/Escritorio/ALL_170/TensorData/Other_tensor_305_S_6899.pt
[941_S_4036] ➔ columnas vacías detectadas y eliminadas: [34, 35, 80, 81]
[941_S_4036] señal tras limpieza: T=197, N=166
 corr: (166, 166)
    → T = 197 muestras; B = 14 bins para MI
 nmi: (166, 166)


Extracting features: 100%|█████████▉| 431/432 [1:03:29<00:09,  9.12s/it]

 gc: (166, 166)
 dcv: (166, 166)
[941_S_4036] guardado en /home/diego/Escritorio/ALL_170/TensorData/Other_tensor_941_S_4036.pt
[941_S_4187] ➔ columnas vacías detectadas y eliminadas: [34, 35, 80, 81]
[941_S_4187] señal tras limpieza: T=197, N=166
 corr: (166, 166)
    → T = 197 muestras; B = 14 bins para MI
 nmi: (166, 166)


Extracting features: 100%|██████████| 432/432 [1:03:38<00:00,  8.84s/it]

 gc: (166, 166)
 dcv: (166, 166)
[941_S_4187] guardado en /home/diego/Escritorio/ALL_170/TensorData/Other_tensor_941_S_4187.pt
